In [2]:
pip install selenium pandas


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()
url = "https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021"
driver.get(url)

main_section = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "cb-nav-main"))
)

title = main_section.find_element(By.CSS_SELECTOR, "h1[itemprop='name']").text.strip()

subhdr = main_section.find_element(By.CLASS_NAME, "cb-nav-subhdr")
links = subhdr.find_elements(By.TAG_NAME, "a")

series_name = links[0].text.strip()
venue = links[1].text.strip()

match_id = url.split('/')[-2]
title_main = title.split(" - ")[0].strip()
match_no = title_main.split(',')[-1].strip() if ',' in title_main else ""

date_span = subhdr.find_element(By.CSS_SELECTOR, 'span[itemprop="startDate"]')
year = date_span.get_attribute("content")[:4] if date_span else ""

series_type = "IPL" if "Premier League" in series_name else "Unknown"
match_type = "T20"

try:
    status_el = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, ".cb-scrcrd-status.cb-text-complete"))
    )
    match_status = status_el.text.strip()
    
    # Extract the match winning team
    if "won" in match_status:
        match_winning_team = match_status.split(" won")[0].strip()
    else:
        match_winning_team = ""
except:
    match_status = ""
    match_winning_team = ""

try:
    commentary_block = driver.find_elements(By.CSS_SELECTOR, "p.cb-com-ln.ng-binding")
    match_tie_breaker = ""
    for para in commentary_block:
        if "super over" in para.text.lower() or "dls" in para.text.lower():
            match_tie_breaker = para.text.strip()
            break
except:
    match_tie_breaker = ""

match_details = {
    "year": year,
    "series_type": series_type,
    "series_name": series_name,
    "match_no": match_no,
    "match_type": match_type,
    "match_id": match_id,
    "match_venue": venue,
    "match_status": match_status,
    "match_winning_team": match_winning_team,
    "match_tie_breaker": match_tie_breaker,
}

csv_filename = "match_details.csv"
header = ["year", "series_type", "series_name", "match_no", "match_type", "match_id", "match_venue", "match_status", "match_winning_team", "match_tie_breaker"]

with open(csv_filename, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=header)
    writer.writeheader()  
    writer.writerow(match_details) 

print(f"Match details saved to {csv_filename}")

driver.quit()


Match details saved to match_details.csv
